In [ ]:
# =============================================================================
# Setup: Import Libraries and API Credentials
# =============================================================================

import pandas as pd
import numpy as np
import requests
from dotenv import load_dotenv
import sys 
from pathlib import Path
import os

# ------------------------------ #
# Must run together to access NASS API key
# ------------------------------ #
src_path = Path.cwd() / 'src'
sys.path.insert(0, str(src_path))
from utils.nass_api import NASS_API_KEY
# ------------------------------ #

In [ ]:
# =============================================================================
# Data Retrieval: Obtain Monthly Corn Prices from NASS API
# =============================================================================

base_url = "https://quickstats.nass.usda.gov/api/api_GET/"

# Request national-level monthly corn prices received by farmers
params = {'key': NASS_API_KEY,
          'source_desc': 'SURVEY',
          'sector_desc': 'CROPS',
          'group_desc': 'FIELD CROPS',
          'commodity_desc': 'CORN',
          'statisticcat_desc': 'PRICE RECEIVED',
          'short_desc': 'CORN, GRAIN - PRICE RECEIVED, MEASURED IN $ / BU',
          'agg_level_desc': 'NATIONAL'
         }

response = requests.get(base_url, params=params)

corn_price = response.json()

corn_price = pd.DataFrame(corn_price['data'])

## Data Cleaning: Select Variables and Remove Missing/Suppressed Values

In [ ]:
# Select relevant columns for analysis
select_vars = ['source_desc', 'sector_desc', 'group_desc', 'commodity_desc', 'statisticcat_desc', 'short_desc', 'domain_desc', 'class_desc', 'state_name',  'year', 'reference_period_desc', 'begin_code', 'freq_desc', 'unit_desc',  'Value']

corn_price_sel = corn_price.loc[:, select_vars]

In [ ]:
# Remove rows with suppressed values: (D) = withheld to avoid disclosing data, (Z) = less than half the unit shown
mask = corn_price_sel['Value'].astype(str).str.contains(r'^\s*\((?:D|Z)\)\s*$', regex=True, na=False)

if mask.any():
    print(f'Removing {np.sum(mask)} rows with suppressed values')
    corn_price_sel = corn_price_sel[~mask]
else: 
    print(f'There are {np.sum(mask)} masked values')

In [ ]:
# Convert Value column to numeric (handles any remaining non-numeric entries)
corn_price_sel['Value'] = pd.to_numeric(corn_price_sel['Value'], errors='coerce')

In [ ]:
# Remove annual average values, keeping only monthly and marketing year observations
# YEAR = annual average across all months (we want individual months)
# Keeps: JAN, FEB, ..., DEC, MARKETING YEAR
corn_price_sel = (
    corn_price_sel
    .query("reference_period_desc != 'YEAR'")
)

## Data Validation: Check for Missing Years and Incomplete Months


### Check for missing years in the dataset

In [ ]:
# Get the years present in the data
actual_years = set(corn_price_sel['year'].astype(int).unique())

# Create the full range of expected years
expected_years = set(range(min(actual_years), max(actual_years)+1))

# Find missing years
missing_years = expected_years - actual_years

print(f"Missing years: {sorted(missing_years)}")
print(f"Total missing years: {len(missing_years)}")

# Also show the range of years we actually have
print(f"Years available: {min(actual_years)} to {max(actual_years)}")
print(f"Total years available: {len(actual_years)}")

### Check that each year contains 13 observations (12 months + marketing year)

In [ ]:
# Count observations per year (expect 13: JAN-DEC + MARKETING YEAR)
count_by_month = (
        corn_price_sel
        .groupby('year')['reference_period_desc']
        .size()
        .reset_index(name='count')
    )

Only the most recent year, 2025, does not contain a full set of 12 months of price data. 

In [ ]:
# Identify years with incomplete data (not equal to 13 observations)
not_eq_12 = (count_by_month['count'] != 13)
idx = count_by_month[not_eq_12].index
count_by_month.loc[idx]

## Data Formatting: Rename Columns and Check for Missing Values

### Rename period and month identifier columns for clarity

In [ ]:
# Rename for readability: reference_period_desc -> month, begin_code -> month_id
corn_price_sel = corn_price_sel.rename(columns={'reference_period_desc': 'month', 
                               'begin_code': 'month_id'})
corn_price_sel['month_id'] = corn_price_sel['month_id'].astype(int)

In [ ]:
# Check for any remaining missing values after cleaning
print(f'''There are {corn_price_sel['Value'].isna().sum()} missing values in the 'Value' column.''')

### Rename Value to nominal_price for semantic clarity

In [ ]:
# Rename Value -> nominal_price to explicitly indicate these are not inflation-adjusted
corn_price_sel = corn_price_sel.rename(columns={'Value': 'nominal_price'})

## Export: Save Cleaned Data to CSV

In [ ]:
# Save cleaned data to CSV file in data directory
output_path = Path('data') / 'corn_prices_year_and_monthly_nass.csv'
corn_price_sel.to_csv(output_path, index=False)